In [1]:
import numpy as np
from construct import PaddedString, Int16un, Struct, Int32sn, Int32un, Int8un, Array
from oct_converter.image_types import OCTVolumeWithMetaData, FundusImageWithMetaData
from pathlib import Path

class E2E(object):

    def __init__(self, filepath):
        self.filepath = Path(filepath)
        if not self.filepath.exists():
            raise FileNotFoundError(self.filepath)
        self.header_structure = Struct(
            'magic1' / PaddedString(12, 'ascii'),
            'version' / Int32un,
            'unknown' / Array(10, Int16un)
        )
        self.main_directory_structure = Struct(
            'magic2' / PaddedString(12, 'ascii'),
            'version' / Int32un,
            'unknown' / Array(10, Int16un),
            'num_entries' / Int32un,
            'current' / Int32un,
            'prev' / Int32un,
            'unknown3' / Int32un,
        )
        self.sub_directory_structure = Struct(
            'pos' / Int32un,
            'start' / Int32un,
            'size' / Int32un,
            'unknown' / Int32un,
            'patient_id' / Int32un,
            'study_id' / Int32un,
            'series_id' / Int32un,
            'slice_id' / Int32sn,
            'unknown2' / Int16un,
            'unknown3' / Int16un,
            'type' / Int32un,
            'unknown4' / Int32un,
        )
        self.chunk_structure = Struct(
            'magic3' / PaddedString(12, 'ascii'),
            'unknown' / Int32un,
            'unknown2' / Int32un,
            'pos' / Int32un,
            'size' / Int32un,
            'unknown3' / Int32un,
            'patient_id' / Int32un,
            'study_id' / Int32un,
            'series_id' / Int32un,
            'slice_id' / Int32sn,
            'ind' / Int16un,
            'unknown4' / Int16un,
            'type' / Int32un,
            'unknown5' / Int32un,
        )
        self.image_structure = Struct(
            'size' / Int32un,
            'type' / Int32un,
            'unknown' / Int32un,
            'width' / Int32un,
            'height' / Int32un,
        )
        self.lat_structure = Struct(
            'unknown' / Array(14, Int8un),
            'laterality' / Int8un,
            'unknown2' / Int8un
        )

        self.power = pow(2, 10)


    def read_oct_volume(self):
        def _make_lut():
            LUT = []
            for i in range(0,pow(2,16)):
                LUT.append(self.uint16_to_ufloat16(i))
            return np.array(LUT)
        LUT = _make_lut() 
               

        with open(self.filepath, 'rb') as f:
            raw = f.read(36)
            header = self.header_structure.parse(raw)

            raw = f.read(52)
            main_directory = self.main_directory_structure.parse(raw)

            # traverse list of main directories in first pass
            directory_stack = []

            current = main_directory.current
            while current != 0:
                directory_stack.append(current)
                f.seek(current)
                raw = f.read(52)
                directory_chunk = self.main_directory_structure.parse(raw)
                current = directory_chunk.prev

            # traverse in second pass and  get all subdirectories
            chunk_stack = []
            volume_dict = {}
            for position in directory_stack:
                f.seek(position)
                raw = f.read(52)
                directory_chunk = self.main_directory_structure.parse(raw)

                for ii in range(directory_chunk.num_entries):
                    raw = f.read(44)
                    chunk = self.sub_directory_structure.parse(raw)
                    volume_string = '{}_{}_{}'.format(chunk.patient_id, chunk.study_id, chunk.series_id)
                    if volume_string not in volume_dict.keys():
                        volume_dict[volume_string] = chunk.slice_id / 2
                    elif chunk.slice_id / 2 > volume_dict[volume_string]:
                        volume_dict[volume_string] = chunk.slice_id / 2

                    if chunk.start > chunk.pos:
                        chunk_stack.append([chunk.start, chunk.size])

            # initalise dict to hold all the image volumes
            volume_array_dict = {}
            volume_array_dict_additional = {} # for storage of slices not caught by extraction
            for volume, num_slices in volume_dict.items():
                if num_slices > 0:
                    # num_slices + 1 here due to evidence that a slice was being missed off the end in extraction
                    volume_array_dict[volume] = [0] * int(num_slices + 1)

            # traverse all chunks and extract slices
            for start, pos in chunk_stack:
                f.seek(start)
                raw = f.read(60)
                chunk = self.chunk_structure.parse(raw)

                if chunk.type == 11:  # laterality data
                    raw = f.read(20)
                    try:
                        laterality_data = self.lat_structure.parse(raw)
                        if laterality_data.laterality == 82:
                            self.laterality = 'R'
                        elif laterality_data.laterality == 76:
                            self.laterality = 'L'
                    except:
                        self.laterality = None

                if chunk.type == 1073741824:  # image data
                    raw = f.read(20)
                    image_data = self.image_structure.parse(raw)

                    if chunk.ind == 1:  # oct data
                        raw_volume = np.fromfile(f, dtype=np.uint16, count=image_data.height * image_data.width)
                        image = LUT[raw_volume].reshape(image_data.width, image_data.height)
                        image = 256 * pow(image, 1.0 / 2.4)
                        volume_string = '{}_{}_{}'.format(chunk.patient_id, chunk.study_id, chunk.series_id)
                        if volume_string in volume_array_dict.keys():
                            volume_array_dict[volume_string][int(chunk.slice_id / 2) - 1] = image
                        else:
                            # try to capture these additional images
                            if volume_string in volume_array_dict_additional.keys():
                                volume_array_dict_additional[volume_string].append(image)
                            else:
                                volume_array_dict_additional[volume_string] = [image]
                            #print('Failed to save image data for volume {}'.format(volume_string))

            oct_volumes = []
            for key, volume in volume_array_dict.items():
                oct_volumes.append(OCTVolumeWithMetaData(volume=volume, patient_id=key, laterality=self.laterality))
            for key, volume in volume_array_dict_additional.items():
                oct_volumes.append(OCTVolumeWithMetaData(volume=volume, patient_id=key, laterality=self.laterality))

        return oct_volumes

    def read_fundus_image(self):
        with open(self.filepath, 'rb') as f:
            raw = f.read(36)
            header = self.header_structure.parse(raw)

            raw = f.read(52)
            main_directory = self.main_directory_structure.parse(raw)

            # traverse list of main directories in first pass
            directory_stack = []

            current = main_directory.current
            while current != 0:
                directory_stack.append(current)
                f.seek(current)
                raw = f.read(52)
                directory_chunk = self.main_directory_structure.parse(raw)
                current = directory_chunk.prev

            # traverse in second pass and  get all subdirectories
            chunk_stack = []
            for position in directory_stack:
                f.seek(position)
                raw = f.read(52)
                directory_chunk = self.main_directory_structure.parse(raw)

                for ii in range(directory_chunk.num_entries):
                    raw = f.read(44)
                    chunk = self.sub_directory_structure.parse(raw)
                    if chunk.start > chunk.pos:
                        chunk_stack.append([chunk.start, chunk.size])

            # initalise dict to hold all the image volumes
            image_array_dict = {}

            # traverse all chunks and extract slices
            for start, pos in chunk_stack:
                f.seek(start)
                raw = f.read(60)
                chunk = self.chunk_structure.parse(raw)

                if chunk.type == 11:  # laterality data
                    raw = f.read(20)
                    try:
                        laterality_data = self.lat_structure.parse(raw)
                        if laterality_data.laterality == 82:
                            self.laterality = 'R'
                        elif laterality_data.laterality == 76:
                            self.laterality = 'L'
                    except:
                        self.laterality = None

                if chunk.type == 1073741824:  # image data
                    raw = f.read(20)
                    image_data = self.image_structure.parse(raw)

                    if chunk.ind == 0:  # fundus data
                        raw_volume = np.fromstring(f.read(image_data.height * image_data.width), dtype=np.uint8)
                        image = np.array(raw_volume).reshape(image_data.height,image_data.width)
                        image_string = '{}_{}_{}'.format(chunk.patient_id, chunk.study_id, chunk.series_id)
                        image_array_dict[image_string] = image


            fundus_images = []
            for key, image in image_array_dict.items():
                fundus_images.append(FundusImageWithMetaData(image=image, patient_id=key, laterality= self.laterality))

        return fundus_images

    def read_custom_float(self, bytes):

        # convert two bytes to 16-bit binary representation
        bits = bin(bytes[0])[2:].zfill(8)[::-1] + bin(bytes[1])[2:].zfill(8)[::-1]

        # get mantissa and exponent
        mantissa = bits[:10]
        exponent = bits[10:]

        # convert to decimal representations
        mantissa_sum = 1 + int(mantissa, 2) / self.power
        exponent_sum = int(exponent[::-1], 2) - 63
        decimal_value = mantissa_sum * pow(2, exponent_sum)
        return decimal_value

    def uint16_to_ufloat16(self, uint16):

        bits = '{0:016b}'.format(uint16)[::-1]
        # get mantissa and exponent
        mantissa = bits[:10]
        exponent = bits[10:]
        exponent = exponent[::-1]

        # convert to decimal representations
        mantissa_sum = 1 + int(mantissa, 2) / self.power
        exponent_sum = int(exponent, 2) - 63
        decimal_value = mantissa_sum * np.float_power(2, exponent_sum)
        return decimal_value

In [2]:
import os
from PIL import Image
import numpy as np

def saveImages():
    # Create the 'images' directory if it doesn't exist
    if not os.path.exists('images'):
        os.makedirs('images')
    
    # Save OCT volume images
    for i, oct_volume in enumerate(oct_volumes):
        if i != 0:
            break
        for j, image in enumerate(oct_volume.volume):
            if image is not None:
                # Convert the image to uint8 (from float) for saving
                image_uint8 = (image / np.max(image) * 255).astype(np.uint8)
                img = Image.fromarray(image_uint8)
                img.save(f'images/oct_volume_{i}_slice_{j}.png')
    
    print("Images saved successfully!")

In [3]:
import os
import numpy as np
from PIL import Image
import tkinter as tk
from tkinter import filedialog, messagebox
import vtk
from skimage import io, transform, measure, filters
import concurrent.futures

# Create the 'images' directory if it doesn't exist
if not os.path.exists('images'):
    os.makedirs('images')

class E2E:
    """Placeholder class for handling .e2e file operations"""
    
    def __init__(self, file_path):
        self.file_path = file_path

    def read_oct_volume(self):
        # Placeholder for actual OCT volume reading logic
        print(f"Reading OCT volume from {self.file_path}")
        return np.random.rand(100, 100, 100)  # Dummy data for demonstration

    def read_fundus_image(self):
        # Placeholder for actual fundus image reading logic
        print(f"Reading fundus images from {self.file_path}")
        return np.random.rand(512, 512)  # Dummy data for demonstration


class ImageProcessor:

    @staticmethod
    def process_folder(folder_path, scale_factor, z_scale_factor):

        npz_dir = "NPZ_files"
        os.makedirs(npz_dir, exist_ok=True)

        file_paths = ImageProcessor._get_image_files(folder_path)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            slices = list(executor.map(lambda f: ImageProcessor._load_and_rescale_image(f, scale_factor), file_paths))

        slices = np.array(slices)

        # Adjust the sigma value to preserve more details
        slices = filters.gaussian(slices, sigma=1.0)

        # Get the min and max intensity values for dynamic thresholding
        min_intensity = np.min(slices)
        max_intensity = np.max(slices)

        # Set a dynamic threshold as a fraction of the intensity range
        threshold = min_intensity + 0.5 * (max_intensity - min_intensity)

        spacing = (z_scale_factor, 1.0, 1.0)

        # Perform marching cubes within the intensity range
        vertices, faces, _, _ = measure.marching_cubes(slices, level=threshold, spacing=spacing)

        folder_name = os.path.basename(folder_path)
        npz_file_path = os.path.join(npz_dir, f"{folder_name}.npz")
        np.savez(npz_file_path, vertices=vertices, faces=faces, slices=slices)

        return npz_file_path

    @staticmethod
    def _load_and_rescale_image(file_path, scale_factor):

        img = io.imread(file_path, as_gray=True)
        return transform.rescale(img, scale_factor, anti_aliasing=True)

    @staticmethod
    def _get_image_files(folder_path):

        file_paths = [os.path.join("images", filename)
                      for filename in sorted(os.listdir("images"))
                      if filename.endswith(".png")]

        if not file_paths:
            raise ValueError("No PNG images found in the selected folder.")
        return file_paths


class Visualizer:

    @staticmethod
    def visualize_3d(npz_file_path, slice_start, slice_end):
        try:
            data = np.load(npz_file_path)
            vertices = data['vertices']
            faces = data['faces']
            slices = data['slices']

            # Limit the slices to the specified range
            limited_slices = slices[slice_start:slice_end + 1]
            min_intensity = np.min(limited_slices)
            max_intensity = np.max(limited_slices)

            # Create the 3D model for the selected slices
            spacing = (1.0, 1.0, 1.0)
            threshold = min_intensity + 0.5 * (max_intensity - min_intensity)

            vertices, faces, _, _ = measure.marching_cubes(limited_slices, level=threshold, spacing=spacing)

            points = vtk.vtkPoints()
            for vertex in vertices:
                points.InsertNextPoint(vertex)

            polydata = vtk.vtkPolyData()
            polydata.SetPoints(points)

            faces_array = vtk.vtkCellArray()
            for face in faces:
                triangle = vtk.vtkTriangle()
                for i in range(3):
                    triangle.GetPointIds().SetId(i, face[i])
                faces_array.InsertNextCell(triangle)

            polydata.SetPolys(faces_array)

            # Add color mapping to enhance visualization
            mapper = vtk.vtkPolyDataMapper()
            mapper.SetInputData(polydata)
            mapper.ScalarVisibilityOn()  # Enable scalar visibility for color mapping

            # Create a lookup table to assign colors to different scalar values
            lut = vtk.vtkLookupTable()
            lut.SetTableRange(vertices.min(), vertices.max())
            lut.Build()

            mapper.SetLookupTable(lut)
            mapper.SetScalarRange(vertices.min(), vertices.max())

            actor = vtk.vtkActor()
            actor.SetMapper(mapper)

            renderer = vtk.vtkRenderer()
            renderer.AddActor(actor)

            render_window = vtk.vtkRenderWindow()
            render_window.AddRenderer(renderer)

            interactor = vtk.vtkRenderWindowInteractor()
            interactor.SetRenderWindow(render_window)

            renderer.ResetCamera()
            render_window.Render()
            interactor.Start()
        except Exception as e:
            raise RuntimeError(f"Error during visualization: {e}")


class ImageProcessingApp:
    """Main application for image processing with a Tkinter GUI."""

    def __init__(self, master):
        self.master = master
        master.title("3D Image Processor")

        self._init_ui()

        self.e2e_file_path = None
        self.npz_file_path = None

    def _init_ui(self):
        """Initializes the UI components."""
        self.scale_label = tk.Label(self.master, text="Scale Factor (XY):")
        self.scale_label.pack()
        self.scale_slider = tk.Scale(self.master, from_=0.1, to=1.0, resolution=0.01, orient="horizontal")
        self.scale_slider.set(0.25)
        self.scale_slider.pack()

        self.z_scale_label = tk.Label(self.master, text="Z-Scale Factor:")
        self.z_scale_label.pack()
        self.z_scale_slider = tk.Scale(self.master, from_=0.5, to=5.0, resolution=0.1, orient="horizontal")
        self.z_scale_slider.set(1.0)
        self.z_scale_slider.pack()

        self.slice_start_label = tk.Label(self.master, text="Start Slice:")
        self.slice_start_label.pack()
        self.slice_start_slider = tk.Scale(self.master, from_=0, to=24, orient="horizontal")
        self.slice_start_slider.set(0)
        self.slice_start_slider.pack()

        self.slice_end_label = tk.Label(self.master, text="End Slice:")
        self.slice_end_label.pack()
        self.slice_end_slider = tk.Scale(self.master, from_=0, to=24, orient="horizontal")
        self.slice_end_slider.set(24)
        self.slice_end_slider.pack()

        self.e2e_button = tk.Button(self.master, text="Select E2E File", command=self.select_e2e_file)
        self.e2e_button.pack()

        self.process_button = tk.Button(self.master, text="Process Images", command=self.process_images)
        self.process_button.pack()

    def select_e2e_file(self):
        """Handles .e2e file selection."""
        self.e2e_file_path = filedialog.askopenfilename(filetypes=[("E2E files", "*.E2E")])
        if self.e2e_file_path:
            messagebox.showinfo("File Selected", f"Selected file: {self.e2e_file_path}")

    def process_images(self):
        """Processes images and visualizes them in 3D."""
        if not self.e2e_file_path:
            messagebox.showwarning("No E2E File", "Please select an E2E file first.")
            return

        scale_factor = self.scale_slider.get()
        z_scale_factor = self.z_scale_slider.get()

        try:
            e2e = E2E(self.e2e_file_path)
            oct_volumes = e2e.read_oct_volume()  # Reading OCT volume
            fundus_images = e2e.read_fundus_image()  # Reading fundus images

            self.npz_file_path = ImageProcessor.process_folder(self.e2e_file_path, scale_factor, z_scale_factor)
            messagebox.showinfo("Processing Complete", f"Saved processed data to: {self.npz_file_path}")

            # Get slice start and end from sliders
            slice_start = self.slice_start_slider.get()
            slice_end = self.slice_end_slider.get()

            # Call visualization for the selected slice range
            Visualizer.visualize_3d(self.npz_file_path, slice_start, slice_end)
        except Exception as e:
            messagebox.showerror("Processing Error", str(e))


if __name__ == "__main__":
    root = tk.Tk()
    app = ImageProcessingApp(root)
    root.mainloop()


C:\Users\Moji\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


Reading OCT volume from C:/Users/Moji/Desktop/Acél Ede/ACEL01E.E2E
Reading fundus images from C:/Users/Moji/Desktop/Acél Ede/ACEL01E.E2E
